In [2]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
!pip install pandas

  Using cached pandas-2.2.2-cp312-cp312-macosx_11_0_arm64.whl.metadata (19 kB)
  Using cached numpy-2.0.0-cp312-cp312-macosx_14_0_arm64.whl.metadata (60 kB)
  Using cached pytz-2024.1-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2024.1-py2.py3-none-any.whl.metadata (1.4 kB)
Using cached pandas-2.2.2-cp312-cp312-macosx_11_0_arm64.whl (11.3 MB)
Using cached numpy-2.0.0-cp312-cp312-macosx_14_0_arm64.whl (5.0 MB)
Using cached pytz-2024.1-py2.py3-none-any.whl (505 kB)
Using cached tzdata-2024.1-py2.py3-none-any.whl (345 kB)

[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: pip install --upgrade pip


In [7]:
import pandas as pd 
import os 
import numpy as np 
import sys  
#sys.path.append("../")
#from utils import analysis_utils, git_utils

In [8]:
outputdir = "../output/evaluation/processed"
projects = ['Lang', 'Math', 'Time', 'Closure', 'Cli', 'Compress', 'Codec', 'Collections', 'Csv', 
    'JacksonCore', 'JacksonXml', 'JxPath', 'Jsoup']

# RQ1. Prevalence

### Prevalence of lived mutants

In [11]:
def get_summarised_statistic_simple(df:pd.DataFrame, project:str):
    def compute_perc(a, b, decimal_point):
        return np.round(100*a/b, decimals=decimal_point) if b != 0 else 0
    columns = ['all', 'survived', 'killed', 'no_covg', 'timed_out', 'non_viable', 'memory_error', 'run_error']
    line = ""
    decimal_point = 2    
    df = df[~(df['all'] == 0)]
    n_all = df['all'].sum() 
    for col in columns[:3]:
        # agg
        if col != 'all': # if not all, compute the percentage
            n_sum = compute_perc(df[col].sum(), n_all, decimal_point)
        else:
            n_sum = n_all #df[col].sum()
        # avg
        if col != 'all':
            vs = df[col].div(df['all']).values 
            vs[(vs == np.inf) | (vs == -np.inf)] = 0.
            n_mean = np.round(np.mean(100*vs), decimals=decimal_point)
        else:
            n_mean = np.round(df[col].mean(), decimals=decimal_point)
        
        sum_stat = f"{n_sum}"# \%" 
        mean_stat = f"{n_mean}"
        #line += f" & {mean_stat} & {sum_stat}"   
        line += f" & {sum_stat}"   # without avg
    n_sum, n_mean = 0, 0
    for col in columns[3:]:
        # agg 
        n_sum += df[col].sum()
        # avg 
        vs = df[col].div(df['all']).values 
        vs[vs == np.inf] = 0.
        n_mean += np.round(np.mean(100*vs), decimals=decimal_point)
    # agg 
    n_sum = np.round(100 * n_sum/df['all'].sum(), decimals=decimal_point)
    sum_stat = f"{n_sum}"
    # avg 
    n_mean = np.round(n_mean, decimals=decimal_point)
    mean_stat = f"{n_mean}" #\%" 
    #line += f" & {mean_stat} & {sum_stat}"
    line += f" & {sum_stat}"
    line += " \\\\"
    print (project + f" & {len(df)}" + line) 
 

In [12]:
dfs = []
for i, project in enumerate(projects):
    df = pd.read_csv(os.path.join(outputdir, f"{project}.init_pit_mut_status.csv"))
    dfs.append(df)
    get_summarised_statistic_simple(df, project)
get_summarised_statistic_simple(pd.concat(dfs), 'Total')

Lang & 49 & 16973 & 7.12 & 56.04 & 36.84 \\
Math & 104 & 28548 & 10.05 & 61.31 & 28.63 \\
Time & 20 & 3086 & 8.85 & 84.54 & 6.61 \\
Closure & 139 & 33969 & 10.07 & 74.26 & 15.66 \\
Cli & 30 & 2760 & 9.93 & 85.25 & 4.82 \\
Compress & 45 & 8371 & 16.9 & 67.69 & 15.41 \\
Codec & 18 & 3561 & 10.11 & 83.46 & 6.43 \\
Collections & 2 & 436 & 14.91 & 19.72 & 65.37 \\
Csv & 14 & 1433 & 12.35 & 80.88 & 6.77 \\
JacksonCore & 22 & 19536 & 14.78 & 61.61 & 23.61 \\
JacksonXml & 3 & 298 & 8.39 & 55.37 & 36.24 \\
JxPath & 9 & 1705 & 9.21 & 72.43 & 18.36 \\
Jsoup & 92 & 10632 & 15.17 & 73.78 & 11.05 \\
Total & 547 & 131308 & 11.23 & 67.3 & 21.47 \\


### Prevalence of latent mutants

In [27]:
without_avg = False

In [28]:
n_dfs_thr = {}
for project in projects:
    mut_propa_stat_debt_file = os.path.join(outputdir, f"{project}.indv_mut_propagation_status_and_debt.csv")
    df = pd.read_csv(mut_propa_stat_debt_file)
    df = df.loc[~((df.status == 'reveal') & (df.debt_time > 365))]
    df = df.loc[~((df.status == 'surv') & (df.debt_time <= 365))]
    rows = []
    for bid, adf in df.groupby('bid'):
        n_surv = (adf.status == 'non-latent').sum()
        n_reveal = (adf.status == 'latent').sum()
        n_dead = (adf.status == 'discard').sum()
        n_all = len(adf)
        rows.append([bid, n_surv, n_reveal, n_dead, n_all])
    n_df = pd.DataFrame(rows, columns=['bid', 'n_non_latent', 'n_latent', 'n_discard', 'n_all'])
    n_df['project'] = [project] * len(n_df)
    n_dfs_thr[project] = n_df

In [29]:
def get_n_changes(df:pd.DataFrame, k = 'bid'):
    df = df[[k, 'survived']]
    n_rev = df[k].shape[0]
    df = df.loc[df.survived > 0]
    n_rev_w_survived = df[k].shape[0]
    total_surv_n = df.survived.sum()
    ret = [n_rev, n_rev_w_survived, total_surv_n]
    ret = [n_rev, df.shape[0], df.survived.sum()]
    return ret, df

decimal_point = 2
dfs = []
n_chg_pproj = {}
val_inval_muts_dfs = []
merged_dfs = []
for project in projects:
    df = pd.read_csv(os.path.join(outputdir, f"{project}.init_pit_mut_status.csv"))
    ret, merged_df = get_n_changes(df)
    merged_df['project'] = [project] * len(merged_df)
    merged_dfs.append(merged_df)
    n_chg_pproj[project] = ret 
    df['p_b_id'] = [f"{project}_{bid}" for bid in df.bid.values] 
    dfs.append(df)
n_chg_pproj['Total'], _ = get_n_changes(pd.concat(dfs), k = 'p_b_id')


In [30]:
def status_distr(project, n_df, n_chg):
    total_n = n_df.n_all.sum()
    
    total_n_non_latent = n_df.n_non_latent.sum()
    total_n_latent = n_df.n_latent.sum()
    total_n_discard = n_df.n_discard.sum()

    perc_surv = np.round(100*total_n_non_latent/total_n, decimals=1)
    perc_dead = np.round(100*total_n_discard/total_n, decimals=1)
    perc_reveal = np.round(100*total_n_latent/total_n, decimals=1)

    avg_n_non_latent = np.round(n_df.n_non_latent.mean(), decimals=1)
    if not without_avg:
        avg_n_latent = np.round(n_df.n_latent.mean(), decimals=1)
        avg_n_discard = np.round(n_df.n_discard.mean(), decimals=1)

        perc_avg_surv  = np.round(100*(n_df.n_non_latent/n_df.n_all).mean(), decimals=1)
        perc_avg_reveal  = np.round(100*(n_df.n_latent/n_df.n_all).mean(), decimals=1)
        perc_avg_dead  = np.round(100*(n_df.n_discard/n_df.n_all).mean(), decimals=1)
    
    n_bid_w_mut = n_df.loc[n_df.n_all > 0][['project', 'bid']].drop_duplicates().shape[0]
    line = f"{project} & {n_chg[0]}" + " & \\textbf{" + f"{n_bid_w_mut}" + "}" + f"/{n_chg[1]}" +  "& \\textbf{" + f"{total_n}" + "}" + f"/{n_chg[2]}"
    if not without_avg:
        line += f"& {avg_n_latent} ({perc_avg_reveal}\%) & {total_n_latent} ({perc_reveal}\%)" 
        line += f" & {avg_n_non_latent} ({perc_avg_surv}\%) & {total_n_non_latent} ({perc_surv}\%)" 
        line += f" & {avg_n_discard} ({perc_avg_dead}\%) & {total_n_discard} ({perc_dead}\%)"
    else:
        line += f" & {total_n_latent} ({perc_reveal}\%)" 
        line += f" & {total_n_non_latent} ({perc_surv}\%)" 
        line += f" & {total_n_discard} ({perc_dead}\%)"
    
    rows = [project, n_chg[0], (n_bid_w_mut, n_chg[1]), (total_n, n_chg[2])]
    if not without_avg:
        rows.extend([(avg_n_latent, perc_avg_reveal), (total_n_latent, perc_reveal)])
        rows.extend([(avg_n_non_latent, perc_avg_surv), (total_n_non_latent, perc_surv)])
        rows.extend([(avg_n_discard, perc_avg_dead), (total_n_discard, perc_dead)])
    else:
        rows.extend([(total_n_latent, perc_reveal)])
        rows.extend([(total_n_non_latent, perc_surv)])
        rows.extend([(total_n_discard, perc_dead)])       
    print (line + "\\\\")

decimal = 1
n_dfs = {}
for project in projects:
    status_distr(project, n_dfs_thr[project], n_chg_pproj[project])
# total 
status_distr('Total', pd.concat(list(n_dfs_thr.values())), n_chg_pproj['Total'])

Lang & 49 & \textbf{30}/33& \textbf{1025}/1209& 2.6 (8.5\%) & 78 (7.6\%) & 25.7 (76.0\%) & 771 (75.2\%) & 5.9 (15.4\%) & 176 (17.2\%)\\
Math & 104 & \textbf{89}/92& \textbf{2099}/2870& 1.8 (6.2\%) & 163 (7.8\%) & 16.4 (70.7\%) & 1461 (69.6\%) & 5.3 (23.0\%) & 475 (22.6\%)\\
Time & 20 & \textbf{19}/19& \textbf{185}/273& 0.5 (4.7\%) & 9 (4.9\%) & 9.2 (94.7\%) & 175 (94.6\%) & 0.1 (0.7\%) & 1 (0.5\%)\\
Closure & 139 & \textbf{118}/118& \textbf{2866}/3421& 0.5 (1.2\%) & 60 (2.1\%) & 19.2 (79.6\%) & 2265 (79.0\%) & 4.6 (19.2\%) & 541 (18.9\%)\\
Cli & 30 & \textbf{22}/25& \textbf{237}/274& 0.7 (4.9\%) & 15 (6.3\%) & 8.4 (88.5\%) & 184 (77.6\%) & 1.7 (6.5\%) & 38 (16.0\%)\\
Compress & 45 & \textbf{43}/44& \textbf{1201}/1415& 3.2 (14.3\%) & 138 (11.5\%) & 12.7 (51.3\%) & 544 (45.3\%) & 12.1 (34.5\%) & 519 (43.2\%)\\
Codec & 18 & \textbf{16}/18& \textbf{282}/360& 1.5 (5.1\%) & 24 (8.5\%) & 9.2 (53.2\%) & 148 (52.5\%) & 6.9 (41.8\%) & 110 (39.0\%)\\
Collections & 2 & \textbf{2}/2& \textbf{4}/65&

<>:24: SyntaxWarning: invalid escape sequence '\%'
<>:24: SyntaxWarning: invalid escape sequence '\%'
<>:25: SyntaxWarning: invalid escape sequence '\%'
<>:25: SyntaxWarning: invalid escape sequence '\%'
<>:26: SyntaxWarning: invalid escape sequence '\%'
<>:26: SyntaxWarning: invalid escape sequence '\%'
<>:28: SyntaxWarning: invalid escape sequence '\%'
<>:29: SyntaxWarning: invalid escape sequence '\%'
<>:30: SyntaxWarning: invalid escape sequence '\%'
<>:24: SyntaxWarning: invalid escape sequence '\%'
<>:24: SyntaxWarning: invalid escape sequence '\%'
<>:25: SyntaxWarning: invalid escape sequence '\%'
<>:25: SyntaxWarning: invalid escape sequence '\%'
<>:26: SyntaxWarning: invalid escape sequence '\%'
<>:26: SyntaxWarning: invalid escape sequence '\%'
<>:28: SyntaxWarning: invalid escape sequence '\%'
<>:29: SyntaxWarning: invalid escape sequence '\%'
<>:30: SyntaxWarning: invalid escape sequence '\%'
/var/folders/nt/v8rnkwdn4z9_57q0bvyb9l1m0000gn/T/ipykernel_89366/363113128.py:24: 

In [32]:
n_dfs_thr['Lang']

,bid,n_non_latent,n_latent,n_discard,n_all,project
0,1,50,0,0,50,Lang
1,3,49,0,0,49,Lang
2,4,3,0,0,3,Lang
3,7,51,4,1,56,Lang
4,8,9,0,0,9,Lang
5,9,9,0,0,9,Lang
6,10,9,0,0,9,Lang
7,11,8,1,0,9,Lang
8,12,7,4,0,11,Lang
9,13,1,0,0,1,Lang
